In [78]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import pandas as pd
import folium.plugins
import IP2Location

url = 'https://github.com/firehol/blocklist-ipsets'
ext2 = 'netset'
ext1 = 'ipset'
ext3 = 'https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/'
counter = 0
allo=[]
IP2LocObj = IP2Location.IP2Location()
IP2LocObj.open("IP2LOCATION-LITE-DB5_1.BIN")
marker_locations=[]
marker_name=[]

def listf(url,ext = ''):
    page = requests.get(url).text
    soup = BeautifulSoup(page,'html.parser')
    return ['https://raw.githubusercontent.com' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

for file in listf(url, ext2):
    counter = counter +1
    file = file.replace('/blob','')
    print(file)
    df = pd.read_csv(file,sep=";",index_col=False)
    for rows in df.iterrows():
        if '#' not in rows[1][0]:
            rows[1][0] =rows[1][0].split("/")[0]
            file.replace(ext3,'')
            
            allo.append((IP2LocObj.get_all(rows[1][0]),rows[1][0],file))
            
            marker_locations.append((IP2LocObj.get_all(rows[1][0]).latitude,IP2LocObj.get_all(rows[1][0]).longitude))
            
           


https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bbcan177_ms1.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bbcan177_ms3.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/cidr_report_bogons.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/darklist_de.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/datacenters.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/dshield.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/dshield_1d.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/dshield_30d.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/dshield_7d.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/et_block.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/et_dshield.netset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/

In [82]:
counter =0
for file in listf(url, ext1):
    if (counter == 0):
        counter = 1
        continue
    file = file.replace('/blob','')
    print(file)
    df = pd.read_csv(file,sep="OFI",index_col=False)
    for rows in df.iterrows():  
        if '#' not in rows[1][0]:  
            
             allo.append((IP2LocObj.get_all(rows[1][0]),rows[1][0],file)) 

https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/alienvault_reputation.ipset


D:\Users\kosta\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/asprox_c2.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_banjori.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_bebloh.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_c2.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_cl.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_cryptowall.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_dircrypt.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_dyre.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_geodo.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_hesperbot.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bambenek_matsnu.ipset
https://raw.githubusercontent.com/fireh

https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_sip_0_1d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_sip_1_7d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_sip_2_30d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_smtp_0_1d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_spam_0_1d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_spam_1_7d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_sql-attack_0_1d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_sql_0_1d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_ssh-blocklist_0_1d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_ssh-ddos_0_1d.ipset
https://raw.githubusercontent.com/firehol/blocklist-ipsets/master/bi_ssh-ddos_2_30d.ipset
https://raw.githubusercontent.com/firehol/bl

OSError: illegal IP address string passed to inet_pton

In [83]:
import geopandas
import mapboxgl
from geojson import Point,FeatureCollection,Feature
features = []
x = 0
scalerank = 1
for i in allo:
    point = Point (([i[0].longitude,i[0].latitude]))
    features.append(({
      "type": "Feature",
      "properties": {
        "scalerank": scalerank,
        "name": i[0].ip,
        "comment": "haha",
        "name_alt": i[0].ip,
        "lat_y": i[0].latitude,
        "long_x": i[0].longitude,
        "region": i[0].region,
        "subregion": i[0].city,
        "featureclass": ""
      },
      "geometry": {
        "type": "Point",
        "coordinates": [i[0].longitude, i[0].latitude]
      }
    }))
    x = x +1
    if x == 100000:
        break
   
        


In [84]:
feature_collection = FeatureCollection(features)


In [85]:
with open('./SUPERCLUSTER/TEXT/FIXTURES/places.json', 'w') as f: #edit your  path
   json.dump(feature_collection,f,indent=4)